# Recursive URL | 再帰的URLルートディレクトリ以下のすべてのURLを処理して読み込むことがあるかもしれません。> We may want to process load all URLs under a root directory.例えば、[Python 3.9のドキュメント](https://docs.python.org/3.9/)を見てみましょう。> For example, let's look at the [Python 3.9 Document](https://docs.python.org/3.9/).これには、一括で読みたくなるような多くの興味深い子ページがあります。> This has many interesting child pages that we may want to read in bulk.もちろん、`WebBaseLoader`はページのリストを読み込むことができます。> Of course, the `WebBaseLoader` can load a list of pages.しかし、課題は子ページのツリーをたどり、実際にそのリストを組み立てることです！> But, the challenge is traversing the tree of child pages and actually assembling that list!`RecursiveUrlLoader`を使用してこれを行います。> We do this using the `RecursiveUrlLoader`.これにより、特定の子要素を除外したり、エクストラクターをカスタマイズしたりするなど、より柔軟性が得られます。> This also gives us the flexibility to exclude some children, customize the extractor, and more.

# Parameters | パラメータ* url: str, クロール対象のURL。  > url: str, the target url to crawl.* exclude\_dirs: Optional\[str]、除外するウェブページのディレクトリ。  > exclude\_dirs: Optional\[str], webpage directories to exclude.* use\_async: Optional\[bool]は、非同期リクエストを使用するかどうかを指定します。非同期リクエストは通常、大規模なタスクでより速く動作します。ただし、非同期を使用すると、遅延ローディング機能が無効になります（関数は引き続き動作しますが、遅延ローディングは行われません）。デフォルトでは、Falseに設定されています。  > use\_async: Optional\[bool], wether to use async requests, using async requests is usually faster in large tasks. However, async will disable the lazy loading feature(the function still works, but it is not lazy). By default, it is set to False.* extractor: Optional\[Callable\[\[str], str]]は、ウェブページからドキュメントのテキストを抽出する関数で、デフォルトではページをそのまま返します。テキストを抽出するためには、goose3やbeautifulsoupのようなツールの使用を推奨します。デフォルトでは、ページをそのまま返すだけです。  > extractor: Optional\[Callable\[\[str], str]], a function to extract the text of the document from the webpage, by default it returns the page as it is. It is recommended to use tools like goose3 and beautifulsoup to extract the text. By default, it just returns the page as it is.* max\_depth: Optional\[int] = None、クロールする最大の深さです。デフォルトでは2に設定されています。ウェブサイト全体をクロールする必要がある場合は、十分に大きな数値を設定することで対応できます。  > max\_depth: Optional\[int] = None, the maximum depth to crawl. By default, it is set to 2. If you need to crawl the whole website, set it to a number that is large enough would simply do the job.* timeout: Optional\[int] = None, 各リクエストのタイムアウトを秒単位で設定します。デフォルトでは10に設定されています。  > timeout: Optional\[int] = None, the timeout for each request, in the unit of seconds. By default, it is set to 10.* prevent\_outside: Optional\[bool] = None, ルートURLの外部へのクローリングを防止するかどうかを指定します。デフォルトでは、Trueに設定されています。  > prevent\_outside: Optional\[bool] = None, whether to prevent crawling outside the root url. By default, it is set to True.

In [ ]:
from langchain.document_loaders.recursive_url_loader import RecursiveUrlLoader

簡単な例を試してみましょう。> Let's try a simple example.

In [ ]:
from bs4 import BeautifulSoup as Soup

url = "https://docs.python.org/3.9/"
loader = RecursiveUrlLoader(
    url=url, max_depth=2, extractor=lambda x: Soup(x, "html.parser").text
)
docs = loader.load()

In [3]:
docs[0].page_content[:50]

'\n\n\n\n\nPython Frequently Asked Questions — Python 3.'

In [4]:
docs[-1].metadata

{'source': 'https://docs.python.org/3.9/library/index.html',
 'title': 'The Python Standard Library — Python 3.9.17 documentation',
 'language': None}

しかし、完璧なフィルタリングを行うことは難しいため、結果には関連性のないものがいくつか表示されることがあります。必要に応じて、返されたドキュメントに対して自分でフィルタリングを行うことができます。ほとんどの場合、返された結果は十分に良いものです。> However, since it's hard to perform a perfect filter, you may still see some irrelevant results in the results. You can perform a filter on the returned documents by yourself, if it's needed. Most of the time, the returned results are good enough.

LangChainのドキュメントでテストを行います。> Testing on LangChain docs.

In [2]:
url = "https://js.langchain.com/docs/modules/memory/integrations/"
loader = RecursiveUrlLoader(url=url)
docs = loader.load()
len(docs)

8